In [40]:
import pandas as pd
import numpy as np

In [41]:
data=pd.read_csv('/content/spam.csv',encoding_errors='ignore')

In [42]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [44]:
data['Unnamed: 4'].value_counts()

GNT:-)"                                                     2
 just Keep-in-touch\" gdeve.."                              1
 Never comfort me with a lie\" gud ni8 and sweet dreams"    1
 CALL 2MWEN IM BK FRMCLOUD 9! J X\""                        1
 one day these two will become FREINDS FOREVER!"            1
Name: Unnamed: 4, dtype: int64

In [45]:
data.isna().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [46]:
data['email']=data['v2'].astype(str)+"-"+data['Unnamed: 2'].fillna(" ")+'-'+data['Unnamed: 3'].fillna(" ")+'-'+data['Unnamed: 4'].fillna(" ")

In [47]:
df=data.drop(['v2','Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [48]:
df

,v1,email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...- - -
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will _ b going to esplanade fr home?- - -
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [49]:
from gensim.models import Word2Vec as wtv
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
#importing simple preprocess
from gensim.utils import simple_preprocess
processed = df.email.apply(lambda x: simple_preprocess(x))

In [51]:
#import word2vec
from gensim.models import Word2Vec
# create Cbow and skip gram
# cbow model
# window- dist between the target and surrounding words default 5
# min count of words taken
# sg ==1 for skip gram, for cbow  sg=0
cbow_w2v_model = Word2Vec(processed, size =300, window =9, min_count =2, sg=0 )
#skipgram model
#train a word2vec model from the given dataset
skgram_w2v_model = Word2Vec(processed, size = 300, window =9, min_count =2, sg=1)
# vocabulary size
print('skgram model vocabulary size: ', len(skgram_w2v_model.wv.vocab))

skgram model vocabulary size:  3900


In [52]:
#cbow model vocabulary
list(cbow_w2v_model.wv.vocab.items())[:20]

[('go', <gensim.models.keyedvectors.Vocab at 0x7f78a7e33310>),
 ('until', <gensim.models.keyedvectors.Vocab at 0x7f78a7e33610>),
 ('point', <gensim.models.keyedvectors.Vocab at 0x7f78a7e33410>),
 ('crazy', <gensim.models.keyedvectors.Vocab at 0x7f78a7e339d0>),
 ('available', <gensim.models.keyedvectors.Vocab at 0x7f78aa915590>),
 ('only', <gensim.models.keyedvectors.Vocab at 0x7f78a7e471d0>),
 ('in', <gensim.models.keyedvectors.Vocab at 0x7f78a7e47350>),
 ('bugis', <gensim.models.keyedvectors.Vocab at 0x7f78a7e47490>),
 ('great', <gensim.models.keyedvectors.Vocab at 0x7f78a7e33ad0>),
 ('world', <gensim.models.keyedvectors.Vocab at 0x7f78a7e47750>),
 ('la', <gensim.models.keyedvectors.Vocab at 0x7f78a7e47190>),
 ('buffet', <gensim.models.keyedvectors.Vocab at 0x7f78a7e47b10>),
 ('cine', <gensim.models.keyedvectors.Vocab at 0x7f78a7e47590>),
 ('there', <gensim.models.keyedvectors.Vocab at 0x7f78a7e476d0>),
 ('got', <gensim.models.keyedvectors.Vocab at 0x7f78a7e47890>),
 ('wat', <gensim.m

In [53]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
model = Word2Vec(processed)
word_vectors = model.wv
word_vectors.save('vectors.kv')
google_w2vec = KeyedVectors.load('vectors.kv')

In [54]:

def get_embedding_w2v(doc_tokens, pre_trained):
  embeddings = []
  if pre_trained:
    model = google_w2vec
  else:
    model = cbow_w2v_model
  # iterate over tokens  to extract their vectors
  for tok in doc_tokens:
    if tok in model.wv.vocab:
      embeddings.append(model.wv.word_vec(tok))  
    # take mean of individual words to get the vector of statement
    return np.mean(embeddings, axis =0)  

In [56]:
# create X from w2vec
X_w2v_model = processed.apply(lambda x :get_embedding_w2v(x,pre_trained =0) ) #0---our model
#X_w2v_model = pd.DataFrame(list(X_w2v_model))
pd.DataFrame(X_w2v_model)
print('X shape:', X_w2v_model.shape)

X shape: (5572,)
